In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat("ex3data1.mat")
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X = data["X"]
y = data["y"]
X.shape, y.shape

((5000, 400), (5000, 1))

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
y = ohe.fit_transform(y)
y

D:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [5]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [6]:
def forward_propagate(X, theta1, theta2):
    a1 = np.column_stack((np.ones(X.shape[0]),X))
    z2 = a1.dot(theta1.T)
    a2 = np.column_stack((np.ones(sigmoid(z2).shape[0]),sigmoid(z2)))
    z3 = a2.dot(theta2.T)
    h = sigmoid(z3)
    return a1,z2,a2,z3,h

In [7]:
def cost_function(X, y, theta1, theta2, learning_rate,m):
    a1,z2,a2,z3,h = forward_propagate(X, theta1, theta2)
    s = 0
    for i in range(h.shape[0]):
        s += y[i].reshape(1,y[i].shape[0]).dot(np.log(h[i]).T)+(1-y[i].reshape(1,y[i].shape[0])).dot(np.log(1-h[i]).T)
    return (-s/h.shape[0])[0,0] + (learning_rate / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

In [8]:
def back_propagation(X, y, theta1, theta2, learning_rate):
    d_sum1 = np.zeros(theta1.shape)
    d_sum2 = np.zeros(theta2.shape)
    m = X.shape[0]
    for i in range(m):
        a1 = X[i].reshape(1,X[i].shape[0])                       #(1,401)
        a1,z2,a2,z3,h = forward_propagate(a1,theta1,theta2)
        d3 = h - y[i].reshape(1,y[i].shape[0])                   #(1,10)
        d2 = np.multiply(d3.dot(theta2),(np.multiply(a2,(1-a2))))#(1,26)
        d_sum1 = d_sum1 + d2[:,1:].T.dot(a1)                     #(25,401)
        d_sum2 = d_sum2 + d3.T.dot(a2)                           #(10,26)
    D1 = d_sum1/m
    D2 = d_sum2/m
    D1[:,1:] = D1[:,1:] + (theta1[:,1:] * learning_rate)/m
    D2[:,1:] = D2[:,1:] + (theta2[:,1:] * learning_rate)/m
    return D1, D2    
    

In [9]:
layer_size = 25
eps = 0.12
para = 1
learning_rate = 2
theta1 = np.matrix(np.random.rand(layer_size,X.shape[1]+1)*2*eps-eps)
theta2 = np.matrix(np.random.rand(y.shape[1],layer_size+1)*2*eps-eps)
theta_error = 0.0001
for i in range(20):
    D1,D2 = back_propagation(X, y, theta1, theta2, para)
    theta1 = theta1 - learning_rate*D1
    theta2 = theta2 - learning_rate*D2
    
D1,D2 = back_propagation(X, y, theta1, theta2, para)

for i in range(5):
    for j in range(5):
        theta1_plus = theta1.copy()
        theta1_minus = theta1.copy()
        theta1_plus[i,j] = theta1_plus[i,j] + theta_error
        theta1_minus[i,j] = theta1_minus[i,j] - theta_error
        appr_deri1 = (cost_function(X, y, theta1_plus, theta2, para, X.shape[0])
                      - cost_function(X, y, theta1_minus, theta2, para, X.shape[0]))/(2*theta_error)
        deri_error = (D1[i,j] - appr_deri1)/D1[i,j] 
        print(f"error is: {deri_error*100}%")
        
for i in range(5):
    for j in range(5):
        theta2_plus = theta2.copy()
        theta2_minus = theta2.copy()
        theta2_plus[i,j] += theta_error
        theta2_minus[i,j] -= theta_error
        appr_deri2 = (cost_function(X, y, theta1, theta2_plus, para, X.shape[0])
                      - cost_function(X, y, theta1, theta2_minus, para, X.shape[0]))/(2*theta_error)
        deri_error = (D2[i,j] - appr_deri2)/D2[i,j]
        print(f"error is: {deri_error*100}%")

error is: 1.0177163340460553e-06%
error is: 1.0136762791261883e-05%
error is: -2.998043541685539e-06%
error is: 1.2222254063315008e-07%
error is: -5.456791223445047e-06%
error is: -1.075368426804153e-05%
error is: 6.197697437434682e-06%
error is: 1.9044138416867502e-06%
error is: -1.094847672002947e-05%
error is: -1.4520926509747314e-05%
error is: -3.1474121804106733e-07%
error is: 3.355470779848903e-05%
error is: 1.3491777117359438e-05%
error is: 7.02945984840377e-06%
error is: -5.767950382452161e-06%
error is: -5.716542342516964e-07%
error is: 1.437112410681711e-05%
error is: 5.543648086529945e-05%
error is: -1.1018948052713028e-07%
error is: -2.127506717488748e-05%
error is: 5.612255760710197e-06%
error is: 1.409430708958806e-06%
error is: 2.9035235709592495e-06%
error is: 3.364901015149497e-06%
error is: -0.0003385035466635025%
error is: -7.994311531976943e-06%
error is: -8.807730886032986e-08%
error is: -1.3519564626206634e-06%
error is: 5.686917925996858e-08%
error is: -2.4365620

In [10]:
layer_size = 25
eps = 0.12
para = 1
learning_rate = 2
theta1 = np.matrix(np.random.rand(layer_size,X.shape[1]+1)*2*eps-eps)
theta2 = np.matrix(np.random.rand(y.shape[1],layer_size+1)*2*eps-eps)

for i in range(300):
    D1,D2 = back_propagation(X, y, theta1, theta2, para)
    theta1 = theta1 - learning_rate*D1
    theta2 = theta2 - learning_rate*D2
    
cost_function(X, y, theta1, theta2, para, X.shape[0])

0.5630117497848104

In [11]:
theta1

matrix([[-0.09054739, -0.03195653,  0.03943493, ..., -0.08311036,
          0.02393197, -0.10468891],
        [-0.2736935 , -0.00358228, -0.0309187 , ..., -0.00529932,
          0.01569476, -0.07426492],
        [-0.05869237,  0.0947295 , -0.07746876, ..., -0.03046711,
          0.02265709,  0.0944654 ],
        ...,
        [-0.35954524, -0.07991107, -0.04091162, ...,  0.10465612,
         -0.04673379,  0.01391159],
        [ 0.22203842,  0.06049505, -0.04698853, ...,  0.08764937,
         -0.07765452,  0.04210829],
        [-0.2809648 ,  0.03609435, -0.09571498, ..., -0.00443356,
          0.03595538, -0.0254052 ]])

In [12]:
theta2

matrix([[-1.66242951e+00,  9.34747653e-02, -1.40635723e-02,
          5.56865318e-01, -1.94128280e+00, -2.27514315e+00,
          8.11646465e-02, -1.35006961e+00,  1.28487226e+00,
         -3.88936702e-01, -2.47327684e+00, -2.73261451e+00,
          3.86950102e-01,  7.51057961e-01, -3.92871930e-01,
          1.43691816e+00,  1.20365889e+00, -1.80786378e-01,
         -4.19638716e-01,  3.98682102e-02, -9.59488505e-01,
          2.88051914e+00, -3.63910253e-01, -4.56569564e-01,
          1.20125705e+00,  2.54414324e-02],
        [-1.04551155e+00, -3.43264044e-02, -1.68595077e-01,
          3.64875880e-01, -1.99373562e+00,  5.50536254e-01,
         -3.55093260e+00, -2.02995302e+00,  7.72624470e-01,
         -3.00170333e-01,  1.09235431e+00,  2.68442822e-01,
          1.08890994e+00,  2.76884443e+00, -6.98264858e-01,
         -2.95142118e-01, -1.87024867e+00, -2.68157113e-02,
         -1.59618836e+00, -2.07644480e-01, -7.66332271e-01,
         -1.34991517e+00, -1.11305784e+00, -1.56516648e+

In [13]:
a1,z2,a2,z3,h = forward_propagate(X,theta1,theta2)
h

matrix([[4.04435724e-06, 3.61739922e-03, 1.87306420e-03, ...,
         2.20745982e-03, 2.47381845e-03, 9.90061451e-01],
        [5.48474452e-06, 4.41256732e-03, 3.34107398e-03, ...,
         1.84200833e-03, 1.23512424e-03, 9.86776259e-01],
        [1.22337972e-05, 8.52301991e-03, 2.66789083e-03, ...,
         2.02524103e-02, 5.27433857e-03, 9.46140025e-01],
        ...,
        [2.87929033e-02, 1.11136123e-03, 2.85243927e-02, ...,
         3.10816825e-02, 3.83505385e-01, 4.85022700e-06],
        [1.23032568e-03, 5.19006734e-05, 2.23905104e-04, ...,
         1.03794964e-01, 9.07435897e-01, 9.23780760e-05],
        [9.93983939e-05, 5.01869481e-03, 2.42307826e-05, ...,
         2.41140919e-02, 3.26037276e-01, 6.33049879e-02]])

In [14]:
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=int64)

In [15]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data["y"])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.02000000000001%
